In [236]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import pandas as pd
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import ast
folder_path = "data"
single_data = "data/participant_9999.csv"
df_list = []

# Iterate through all CSV files in the folder
for file in os.listdir(folder_path):
    if file.endswith(".csv"):  # Check if the file is a CSV
        file_path = os.path.join(folder_path, file)
        print(f"Reading: {file}")
        try:
            df_individual = pd.read_csv(file_path, on_bad_lines='skip')
            df_list.append(df_individual)
        except Exception as e:
            print(f"Error reading {file}: {e}")

df = pd.concat(df_list, ignore_index=True)
df_individual = pd.read_csv(single_data, on_bad_lines='skip')
df = df_individual # CHANGE
df['ID'] = df['ID'].ffill()
df['ID'] = df['ID'].astype(int)
df['PayoutDistNum'] = df['PayoutDistNum'].ffill()
df['PayoutDistNum'] = df['PayoutDistNum'].astype(int)

trial_df = df[df['TrialType']=='pirate_recog'].dropna(axis=1,how='all')
trial_df['reward'] = trial_df['reward'].astype(int)

trial_df.groupby('ID')['reward'].mean()

Reading: participant_1111.csv
Reading: participant_3007.csv
Reading: participant_3014.csv
Reading: participant_3025.csv
Reading: participant_3026.csv
Reading: participant_9999.csv


ID
9999    0.983333
Name: reward, dtype: float64

In [237]:
df

,ID,TrialType,PayoutDistNum,BlockNum,contextOrder,reward_rate_red,reward_rate_white,reward_rate_black,probe_order,QuizFailedNum,...,context,reward_prob_red,reward_prob_white,reward_prob_black,choice,probe,reward,confidence,TimeInBlock,Bonus
0,9999,InitializeStudy,3,NaN,"['driftwood_beach', 'driftwood_beach', 'driftw...","[0.3, 0.262, 0.393, 0.34, 0.275, 0.317, 0.319,...","[0.6, 0.569, 0.617, 0.635, 0.668, 0.516, 0.563...","[0.1, 0.126, 0.162, 0.132, 0.205, 0.295, 0.059...","['126', '43', '62', '24', '180', '82', '151', ...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9999,Instruction_1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9999,Instruction_2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9999,Instruction_3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9999,Instruction_4,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,9999,best_pirate,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,stonepine_forest,NaN,NaN,NaN,red_pirate,NaN,NaN,NaN,4.351700,NaN
522,9999,second_best_pirate,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,stonepine_forest,NaN,NaN,NaN,white_pirate,NaN,NaN,NaN,1.649318,NaN
523,9999,best_pirate,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,greenrock_mountain,NaN,NaN,NaN,black_pirate,NaN,NaN,NaN,3.112702,NaN
524,9999,second_best_pirate,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,greenrock_mountain,NaN,NaN,NaN,red_pirate,NaN,NaN,NaN,1.720397,NaN


In [238]:
# Step 1: Convert string to actual list
location_list = ast.literal_eval(df_individual['contextOrder'][0])

# Step 2: Get unique items
unique_locations = list(set(location_list))


print(unique_locations)

['stonepine_forest', 'icecap_mountain', 'coral_beach', 'greenrock_mountain', 'sunleaf_forest', 'driftwood_beach']


In [327]:
df_individual['reward'] = pd.to_numeric(df_individual['reward'], errors='coerce')
df_probe = df_individual[df_individual['TrialType']=='pirate_recog'].dropna(axis=1, how='all').reset_index(drop='index')


In [332]:
df_probe = df_probe[0:59]
df_probe

,ID,TrialType,PayoutDistNum,TimeElapsed,key_press,RT,context,choice,reward,confidence,TimeInBlock
0,9999,pirate_recog,3,3168.868983,5.0,1.350887,5,sure_old,1.0,sure,2.436462
1,9999,pirate_recog,3,3175.449082,5.0,1.083412,2,sure_old,1.0,sure,2.165596
2,9999,pirate_recog,3,3182.667642,6.0,1.898473,1,unsure_old,1.0,unsure,2.986470
3,9999,pirate_recog,3,3217.282198,5.0,1.315037,3,sure_old,1.0,sure,2.399760
4,9999,pirate_recog,3,3224.630083,5.0,1.193709,1,sure_old,1.0,sure,2.277047
5,9999,pirate_recog,3,3251.545192,5.0,1.325627,2,sure_old,1.0,sure,2.410717
6,9999,pirate_recog,3,3259.426435,5.0,1.265647,2,sure_old,1.0,sure,2.349073
7,9999,pirate_recog,3,3272.423580,8.0,1.416038,NaN,sure_new,1.0,sure,2.505385
8,9999,pirate_recog,3,3279.088762,5.0,1.299816,5,sure_old,1.0,sure,2.386147
9,9999,pirate_recog,3,3285.753167,5.0,1.247183,3,sure_old,1.0,sure,2.332084


In [ ]:
pt2_df = df_individual[df_individual['TrialType'].str.contains("pt2")].dropna(axis=1,how='all')
pt2_df['TrialNum'] = pt2_df['TrialType'].str.extract(r'_(\d+)')
pt2_df['TrialChoice'] = pt2_df['choice'].str.extract(r'(\w+)_')


,ID,TrialType,PayoutDistNum,TimeElapsed,key_press,RT,context,reward_prob_red,reward_prob_white,reward_prob_black,choice,reward,TimeInBlock,"Deck, Trial"
0,9999,pt2_pirate_15,3,3172.986606,3.0,0.531955,blank,0.316,0.330,0.665,black_pirate,1.0,3.854442,"black, 15"
1,9999,pt2_pirate_16,3,3179.380383,3.0,0.341242,blank,0.292,0.334,0.565,black_pirate,1.0,3.665239,"black, 16"
2,9999,pt2_pirate_17,3,3186.648219,3.0,0.394924,blank,0.192,0.327,0.521,black_pirate,1.0,3.713845,"black, 17"
3,9999,pt2_pirate_24,3,3222.078178,3.0,1.177063,blank,0.414,0.146,0.674,black_pirate,0.0,4.494384,"black, 24"
4,9999,pt2_pirate_25,3,3228.728547,3.0,0.472401,blank,0.470,0.239,0.679,black_pirate,1.0,3.794388,"black, 25"
5,9999,pt2_pirate_31,3,3256.794000,3.0,1.655451,blank,0.446,0.082,0.646,black_pirate,1.0,4.969915,"black, 31"
6,9999,pt2_pirate_32,3,3265.731966,2.0,2.663541,blank,0.539,0.096,0.483,white_pirate,0.0,5.993996,"white, 32"
7,9999,pt2_pirate_34,3,3276.390013,2.0,0.379397,blank,0.297,0.245,0.482,white_pirate,0.0,3.685583,"white, 34"
8,9999,pt2_pirate_35,3,3283.106136,2.0,0.425583,blank,0.312,0.210,0.556,white_pirate,0.0,3.734718,"white, 35"
9,9999,pt2_pirate_36,3,3289.716297,1.0,0.369561,blank,0.363,0.167,0.577,red_pirate,0.0,3.679048,"red, 36"


In [321]:
target_rows = df_individual[df_individual['TrialType'] == 'pirate_recog'].index + 1

valid_indices = target_rows[target_rows < len(df)]

result_df = df_individual.loc[valid_indices].reset_index(drop=True)

result_df = result_df[result_df['TrialType'].str.contains("pt2")].dropna(axis=1,how='all')
result_df['Deck, Trial'] = result_df['choice'].str.extract(r'^(black|red|white)')[0] + ', ' + result_df['TrialType'].str.extract(r'pt2_pirate_(\d+)')[0]
result_df
sub_df = result_df[['ID','TrialType','Deck, Trial','reward']]
sub_df



,ID,TrialType,"Deck, Trial",reward
0,9999,pt2_pirate_15,"black, 15",1.0
1,9999,pt2_pirate_16,"black, 16",1.0
2,9999,pt2_pirate_17,"black, 17",1.0
3,9999,pt2_pirate_24,"black, 24",0.0
4,9999,pt2_pirate_25,"black, 25",1.0
5,9999,pt2_pirate_31,"black, 31",1.0
6,9999,pt2_pirate_32,"white, 32",0.0
7,9999,pt2_pirate_34,"white, 34",0.0
8,9999,pt2_pirate_35,"white, 35",0.0
9,9999,pt2_pirate_36,"red, 36",0.0


In [326]:
col_df  =sub_df['Deck, Trial'].str.extract(r'(\d+)').astype(int)
# Extract the numbers as a list
numbers = col_df[0].tolist()

colors = ["red", "white", "black"]

# Create combined strings
combined = [f"{color}, {num}" for num in numbers for color in colors]

# Create final DataFrame
regression_df = pd.DataFrame(combined, columns=["Deck, Trial"])

# First, split the "Deck, Trial" column into color and number
regression_df[['color', 'number']] = regression_df['Deck, Trial'].str.split(', ', expand=True)
regression_df['number'] = regression_df['number'].astype(int) 

# Now, define a function that checks if there's a match in pt2_df
def compute_di1(row):
    return int(
        ((sub_df['Deck, Trial'] == row['Deck, Trial'])).any()
    )
def compute_di2(row):
    return int(
        ((pt2_df['TrialNum'].astype(int) == row['number']- 1) & 
         (pt2_df['TrialChoice'].str.lower() == row['color'])).any()
    )
def compute_di3(row):
    return int(
        ((pt2_df['TrialNum'].astype(int) == row['number']- 2) & 
         (pt2_df['TrialChoice'].str.lower() == row['color'])).any()
    )
def compute_dr1(row):
    return int(
        ((sub_df['reward'] == 1) &
        (sub_df['Deck, Trial'] == row['Deck, Trial'])).any()
    )
def compute_dr2(row):
    # Filter for matching trial and choice
    match = pt2_df[
        (pt2_df['TrialNum'].astype(int) == row['number']-1) &
        (pt2_df['TrialChoice'].str.lower() == row['color'])
    ]
    # If there's a match and reward == 1, return 1
    if not match.empty and (match['reward'] == 1).any():
        return 1
    return 0
def compute_dr3(row):
    # Filter for matching trial and choice
    match = pt2_df[
        (pt2_df['TrialNum'].astype(int) == row['number']-2) &
        (pt2_df['TrialChoice'].str.lower() == row['color'])
    ]
    # If there's a match and reward == 1, return 1
    if not match.empty and (match['reward'] == 1).any():
        return 1
    return 0


# Apply the function to create the DI column
regression_df['DI-1'] = regression_df.apply(compute_di1, axis=1)
regression_df['DR-1'] = regression_df.apply(compute_dr1, axis=1)
regression_df['DI-2'] = regression_df.apply(compute_di2, axis=1)
regression_df['DR-2'] = regression_df.apply(compute_dr2, axis=1)
regression_df['DI-3'] = regression_df.apply(compute_di3, axis=1)
regression_df['DR-3'] = regression_df.apply(compute_dr3, axis=1)
regression_df

,"Deck, Trial",color,number,DI-1,DR-1,DI-2,DR-2,DI-3,DR-3
0,"red, 15",red,15,0,0,0,0,0,0
1,"white, 15",white,15,0,0,1,0,1,0
2,"black, 15",black,15,1,1,0,0,0,0
3,"red, 16",red,16,0,0,0,0,0,0
4,"white, 16",white,16,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
172,"white, 126",white,126,0,0,0,0,1,0
173,"black, 126",black,126,1,1,1,1,0,0
174,"red, 128",red,128,1,0,1,1,0,0
175,"white, 128",white,128,0,0,0,0,0,0


,"Deck, Trial",color,number,DI
0,"red, 14",red,14,0
1,"white, 14",white,14,1
2,"black, 14",black,14,0
3,"red, 15",red,15,0
4,"white, 15",white,15,0
...,...,...,...,...
172,"white, 125",white,125,0
173,"black, 125",black,125,1
174,"red, 127",red,127,1
175,"white, 127",white,127,0


In [231]:
# Find index of the first occurrence of 'pirate_1'
start_idx = df_individual[df_individual['TrialType'] == 'pt2_pirate_2'].index[0]

# Find index of the first occurrence of 'Instruction_44'
end_idx = df_individual[df_individual['TrialType'] == 'Instruction_50'].index[0]

# Slice the dataframe from start to end (inclusive)
sub_df = df_individual.loc[start_idx:end_idx-1].reset_index(drop=True)
sub_df.dropna(axis=1,how='all')

""


In [232]:
best_pirate_df = df_individual[df_individual['TrialType'] == 'best_pirate'].dropna(axis=1,how='all').reset_index(drop='index')

In [233]:
import ast
import pandas as pd

# Parse context order
chosen_pirates = best_pirate_df['choice'].dropna().values
context_order = ast.literal_eval(df_individual['contextOrder'].dropna().values[0])
unique_contexts = pd.Series(context_order).unique()

# Initialize list to store correctness
correct_flags = []
first_flags = []
for i, c in enumerate(unique_contexts):
    # Filter relevant rows
    df_context = df_individual[
        (df_individual['TrialType'].str.startswith("pirate_")) &
        (df_individual['context'] == c)
    ].dropna(axis=1, how='all')

    # Assign index column
    df_context['index'] = ['first' if j < 10 else 'main' for j in range(len(df_context))]

    # Group and compute means
    group_means = df_context.drop(columns='probe') \
        .groupby('index')[['reward_prob_red', 'reward_prob_white', 'reward_prob_black']] \
        .mean().reset_index()

    # Get max-probability color in "main" trials
    main_row = group_means[group_means['index'] == 'main'][['reward_prob_red', 'reward_prob_white', 'reward_prob_black']]
    max_col = main_row.idxmax(axis=1).values[0].replace("reward_prob_", "")

    # Get actual choice
    chose = chosen_pirates[i].replace("_pirate", "")

    # Append correctness flag
    correct_flags.append(1 if max_col == chose else 0)

    print(f"Context: {c} | Chose: {chose} | Correct: {max_col == chose} | Best Color: {max_col}")


    # Get max-probability color in "main" trials
    main_row = group_means[group_means['index'] == 'first'][['reward_prob_red', 'reward_prob_white', 'reward_prob_black']]
    next_col = main_row.idxmax(axis=1).values[0].replace("reward_prob_", "")

    # Get actual choice
    chose = chosen_pirates[i].replace("_pirate", "")

    # Append correctness flag
    first_flags.append(1 if next_col == chose else 0)

    print(f"Context: {c} | Chose: {chose} | Correct (first ten): {next_col == chose} | Best Color: {next_col}")

# Assign the full list to the DataFrame
best_pirate_df['correct'] = correct_flags
best_pirate_df['first_correct'] = first_flags

Context: driftwood_beach | Chose: white | Correct: False | Best Color: black
Context: driftwood_beach | Chose: white | Correct (first ten): True | Best Color: white
Context: sunleaf_forest | Chose: white | Correct: True | Best Color: white
Context: sunleaf_forest | Chose: white | Correct (first ten): False | Best Color: black
Context: icecap_mountain | Chose: red | Correct: True | Best Color: red
Context: icecap_mountain | Chose: red | Correct (first ten): False | Best Color: white
Context: coral_beach | Chose: black | Correct: True | Best Color: black
Context: coral_beach | Chose: black | Correct (first ten): False | Best Color: red
Context: stonepine_forest | Chose: red | Correct: True | Best Color: red
Context: stonepine_forest | Chose: red | Correct (first ten): False | Best Color: black
Context: greenrock_mountain | Chose: black | Correct: False | Best Color: white
Context: greenrock_mountain | Chose: black | Correct (first ten): False | Best Color: red


In [234]:
best_pirate_df

,TrialType,TimeElapsed,key_press,RT,context,choice,TimeInBlock,correct,first_correct
0,best_pirate,4053.910825,2.0,1.625439,driftwood_beach,white_pirate,2.671070,0,1
1,best_pirate,4065.004759,2.0,2.658656,sunleaf_forest,white_pirate,3.712437,1,0
2,best_pirate,4074.179119,1.0,3.169927,icecap_mountain,red_pirate,4.220569,1,0
3,best_pirate,4081.903655,3.0,3.126842,coral_beach,black_pirate,4.175837,1,0
4,best_pirate,4089.661596,1.0,3.301669,stonepine_forest,red_pirate,4.351700,1,0
5,best_pirate,4096.271224,3.0,2.063107,greenrock_mountain,black_pirate,3.112702,0,0


In [ ]:
max_col==chose

True

In [ ]:
chosen_pirates[0].replace("_pirate","")

'red'